# IRIS-HEP CMS Integration Challenge: Workflow Demonstration

This notebook demonstrates the workflow of the IRIS-HEP CMS integration challenge. The workflow is modular and consists of the following steps:

1. Environment Setup
2. Configuration Construction
3. Metadata Extraction
4. Skimming
5. Histogramming
6. Statistical Analysis
7. Plotting

## 1. Environment Setup

To set up the environment, use the following command to launch JupyterLab with the required dependencies:

```sh
pixi run lab
```

Alternatively, if you prefer a Conda-like environment, activate it using the provided script:

```sh
source pixi_activate.sh
```

In [1]:
# [coffea-casa] Install packages
# !pip install omegaconf
# !pip install coffea==2025.7.3

In [2]:
# Dask stuff
# Register some modules for dask workers to pickle/unpickle
import cloudpickle
import utils, user
cloudpickle.register_pickle_by_value(utils)
cloudpickle.register_pickle_by_value(user)

# Setup some funcitons for dask client handling
from dask.distributed import Client, LocalCluster, PipInstall

COFFEA_CASA = True

def get_client(n_workers=4):
    if not COFFEA_CASA:
        cluster = LocalCluster(n_workers=n_workers, processes=True, threads_per_worker=1)
        client = Client(cluster)
        print(f"Dask dashboard: {cluster.dashboard_link}")
    else:
        dependencies = ["coffea==2025.7.3"]
        client = Client("tls://localhost:8786")
        client.register_plugin(PipInstall(packages=dependencies))
        cluster = None  # no local cluster in this mode
    return client, cluster

def close_client(cluster, client):
    if client:
        client.close()
    if cluster:
        cluster.close()
    return None, None

## 2. Configuration Construction

A lightweight Python config defines general settings, datasets, cuts, observables, and systematics — all are type-checked and (mostly) CLI-overrideable.
Everything else in the workflow reads from it.

In [3]:
# Import Rich-based Configuration Display from logging module
from utils.logging import display_config_table, get_config_logger

# Create a global config logger instance for this notebook
config_logger = get_config_logger()

In [4]:
# Example: Demonstrate configuration comparison and change detection
from user.configuration_demo import config as original_config
from utils.schema import Config, load_config_with_restricted_cli

import copy

# Save the original configuration for comparison
config = copy.deepcopy(original_config)

print("=== Full configuration ===")
display_config_table(config, expand=True)

=== Full configuration ===


                                                 Configuration: GENERAL                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Key                          ┃ Value                                                                             
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ lumi                         │ 16400                                                                             
│ weight_branch                │ 'genWeight'                                                                       
│ analysis                     │ 'nondiff'                                                                         
│ run_skimming                 │ False                                                                             
│ run_histogramming            │ True                                                                              
│ run_statistics               │ True                                                                              
│ run_systematics              │ True                                                                              
│ run_plots_only               │ False                                                                             
│ run_mva_training             │ False                                                                             
│ run_metadata_generation      │ False                                                                             
│ read_from_cache              │ True                                                                              
│ output_dir                   │ 'example-demo/outputs/'                                                           
│ lumifile                     │ './corrections/Cert_271036-284044_13TeV_Legacy2016_Collisions16_JSON.txt'         
│ cache_dir                    │ '/tmp/integration/'                                                               
└──────────────────────────────┴───────────────────────────────────────────────────────────────────────────────────

                                               Configuration: PREPROCESS                                           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Key                           ┃ Value                                                                            
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ branches                      │ {                                                                                
│                               │   Muon:                                                                          
│                               │ [                                                                                
│                               │     [0]: 'pt'                                                                    
│                               │     [1]: 'eta'                                                                   
│                               │     [2]: 'phi'                                                                   
│                               │     [3]: 'mass'                                                                  
│                               │     [4]: 'miniIsoId'                                                             
│                               │     [5]: 'tightId'                                                               
│                               │     [6]: 'charge'                                                                
│                               │   ]                                                                              
│                               │   FatJet: ['particleNet_TvsQCD', 'pt', 'eta', 'phi', 'mass']                     
│                               │   Jet:                                                                           
│                               │ [                                                                                
│                               │     [0]: 'btagDeepB'                                                             
│                               │     [1]: 'jetId'                                                                 
│                               │     [2]: 'pt'                                                                    
│                               │     [3]: 'eta'                                                                   
│                               │     [4]: 'phi'                                                                   
│                               │     [5]: 'mass'                                                                  
│                               │   ]                                                                              
│                               │   PuppiMET: ['pt', 'phi']                                                        
│                               │   HLT: ['TkMu50']                                                                
│                               │   Pileup: ['nTrueInt']                                                           
│                               │   event: ['genWeight', 'run', 'luminosityBlock', 'event']                        
│                               │ }                                                                                
│ ignore_missing                │ False                                                                            
│ mc_branches                   │ {                                                                                
│                               │   event: ['genWeight']                                                           
│                               │   Pileup: ['nTrueInt']                                                           
│                               │ }                     

                                           Configuration: BASELINE_SELECTION                                       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Key                               ┃ Value                                                                        
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ function                          │ Function: Zprime_hardcuts_no_fj                                              
│ use                               │ [('Muon', None), ('Jet', None)]                                              
└───────────────────────────────────┴──────────────────────────────────────────────────────────────────────────────

                                            Configuration: GOOD_OBJECT_MASKS                                       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Key                               ┃ Value                                                                        
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ analysis                          │ [                                                                            
│                                   │   [0]:                                                                       
│                                   │ {                                                                            
│                                   │     object: 'Muon'                                                           
│                                   │     function: Function: <lambda>                                             
│                                   │     use: [('Muon', None)]                                                    
│                                   │   }                                                                          
│                                   │   [1]:                                                                       
│                                   │ {                                                                            
│                                   │     object: 'Jet'                                                            
│                                   │     function: Function: <lambda>                                             
│                                   │     use: [('Jet', None)]                                                     
│                                   │   }                                                                          
│                                   │   [2]:                                                                       
│                                   │ {                                                                            
│                                   │     object: 'FatJet'                                                         
│                                   │     function: Function: <lambda>                                             
│                                   │     use: [('FatJet', None)]                                                  
│                                   │   }                                                                          
│                                   │ ]                                                                            
│ mva                               │ [                                                                            
│                                   │   [0]:                                                                       
│                                   │ {                                                                            
│                                   │     object: 'Muon'                                                           
│                                   │     function: Function: <lambda>                                             
│                                   │     use: [('Muon', None)]                                                    
│                                   │   }                                                                          
│                                   │   [1]:                                                                       
│                                   │ {                                                                            
│                                   │     object: 'FatJet'                                                         
│                                   │     function: Func

...

                                               Configuration: CORRECTIONS                                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Key                               ┃ Value                                                                        
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ Items                             │ [                                                                            
│                                   │   [0]:                                                                       
│                                   │ {                                                                            
│                                   │     name: 'pu_weight'                                                        
│                                   │     file: 'corrections/puWeights.json.gz'                                    
│                                   │     type: 'event'                                                            
│                                   │     use: [('Pileup', 'nTrueInt')]                                            
│                                   │     op: 'mult'                                                               
│                                   │     key: 'Collisions16_UltraLegacy_goldenJSON'                               
│                                   │     use_correctionlib: True                                                  
│                                   │   }                                                                          
│                                   │   [1]:                                                                       
│                                   │ {                                                                            
│                                   │     name: 'muon_id_sf'                                                       
│                                   │     file: 'corrections/muon_Z.json.gz'                                       
│                                   │     use: [('Muon', 'eta'), ('Muon', 'pt')]                                   
│                                   │     transform: Function: <lambda>                                            
│                                   │     type: 'event'                                                            
│                                   │     key: 'NUM_TightID_DEN_TrackerMuons'                                      
│                                   │     use_correctionlib: True                                                  
│                                   │     op: 'mult'                                                               
│                                   │     up_and_down_idx: ['systup', 'systdown']                                  
│                                   │   }                                                                          
│                                   │ ]                                                                            
└───────────────────────────────────┴──────────────────────────────────────────────────────────────────────────────

                                               Configuration: SYSTEMATICS                                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Key                               ┃ Value                                                                        
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ Items                             │ [                                                                            
│                                   │   [0]:                                                                       
│                                   │ {                                                                            
│                                   │     name: 'jet_pt_resolution'                                                
│                                   │     up_function: Function: jet_pt_resolution                                 
│                                   │     target: ('Jet', 'pt')                                                    
│                                   │     use: [('Jet', 'pt')]                                                     
│                                   │     symmetrise: True                                                         
│                                   │     op: 'mult'                                                               
│                                   │     type: 'object'                                                           
│                                   │   }                                                                          
│                                   │   [1]:                                                                       
│                                   │ {                                                                            
│                                   │     name: 'jet_pt_scale'                                                     
│                                   │     up_function: Function: jet_pt_scale                                      
│                                   │     target: ('Jet', 'pt')                                                    
│                                   │     symmetrise: True                                                         
│                                   │     op: 'mult'                                                               
│                                   │     type: 'object'                                                           
│                                   │   }                                                                          
│                                   │ ]                                                                            
└───────────────────────────────────┴──────────────────────────────────────────────────────────────────────────────

                                               Configuration: STATISTICS                                           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Key                             ┃ Value                                                                          
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ cabinetry_config                │ 'example-demo/outputs/cabinetry/cabinetry_config.yaml'                         
└─────────────────────────────────┴────────────────────────────────────────────────────────────────────────────────

                                                Configuration: PLOTTING                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Key                              ┃ Value                                                                         
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ process_colors                   │ {                                                                             
│                                  │   ttbar_semilep: '#907AD6'                                                    
│                                  │   signal: '#DABFFF'                                                           
│                                  │   ttbar_lep: '#7FDEFF'                                                        
│                                  │   ttbar_had: '#2C2A4A'                                                        
│                                  │   wjets: '#72A1E5'                                                            
│                                  │   ttbar: '#907AD6'                                                            
│                                  │ }                                                                             
│ process_labels                   │ {                                                                             
│                                  │   ttbar_semilep: '$t\bar{t}\,\textrm{(lepton+jets)}$'                         
│                                  │   signal: '$Z^{\prime} \rightarrow t\bar{t}$'                                 
│                                  │   ttbar_lep: '$t\bar{t}\,\textrm{(leptonic)}$'                                
│                                  │   ttbar_had: '$t\bar{t}\,\textrm{(hadronic)}$'                                
│                                  │   wjets: '$W+\textrm{jets}$'                                                  
│                                  │   ttbar: '$t\bar{t}$'                                                         
│                                  │ }                                                                             
│ process_order                    │ [                                                                             
│                                  │   [0]: 'ttbar'                                                                
│                                  │   [1]: 'ttbar_had'                                                            
│                                  │   [2]: 'ttbar_lep'                                                            
│                                  │   [3]: 'ttbar_semilep'                                                        
│                                  │   [4]: 'wjets'                                                                
│                                  │   [5]: 'signal'                                                               
│                                  │ ]                                                                             
└──────────────────────────────────┴───────────────────────────────────────────────────────────────────────────────

                                                Configuration: DATASETS                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Key                        ┃ Value                                                                               
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ datasets                   │ [                                                                                   
│                            │   [0]:                                                                              
│                            │ {                                                                                   
│                            │     name: 'signal'                                                                  
│                            │     directory: 'example-demo/cms_datasets/2016/ZPrimeToTT_M2000_W200/'              
│                            │     cross_section: 0.01895                                                          
│                            │     file_pattern: '*.txt'                                                           
│                            │     tree_name: 'Events'                                                             
│                            │     weight_branch: 'genWeight'                                                      
│                            │     redirector: 'root://xcache/'                                                    
│                            │   }                                                                                 
│                            │   [1]:                                                                              
│                            │ {                                                                                   
│                            │     name: 'ttbar_semilep'                                                           
│                            │     directory: 'example-demo/cms_datasets/2016/TTToSemiLeptonic/'                   
│                            │     cross_section: 364.31                                                           
│                            │     file_pattern: '*.txt'                                                           
│                            │     tree_name: 'Events'                                                             
│                            │     weight_branch: 'genWeight'                                                      
│                            │     redirector: 'root://xcache/'                                                    
│                            │   }                                                                                 
│                            │   [2]:                                                                              
│                            │ {                                                                                   
│                            │     name: 'ttbar_had'                                                               
│                            │     directory: 'example-demo/cms_datasets/2016/TTToHadronic/'                       
│                            │     cross_section: 380.11                                                           
│                            │     file_pattern: '*.txt'                                                           
│                            │     tree_name: 'Events'                                                             
│                            │     weight_branch: 'genWeight'                                                      
│                            │     redirector: 'root://xcache/'                                                    
│                            │   }                      

In [5]:
# Let's look at the datasets config
display_config_table({"datasets": config["datasets"]},
                    expand=True)

                                                Configuration: DATASETS                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Key                        ┃ Value                                                                               
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ datasets                   │ [                                                                                   
│                            │   [0]:                                                                              
│                            │ {                                                                                   
│                            │     name: 'signal'                                                                  
│                            │     directory: 'example-demo/cms_datasets/2016/ZPrimeToTT_M2000_W200/'              
│                            │     cross_section: 0.01895                                                          
│                            │     file_pattern: '*.txt'                                                           
│                            │     tree_name: 'Events'                                                             
│                            │     weight_branch: 'genWeight'                                                      
│                            │     redirector: 'root://xcache/'                                                    
│                            │   }                                                                                 
│                            │   [1]:                                                                              
│                            │ {                                                                                   
│                            │     name: 'ttbar_semilep'                                                           
│                            │     directory: 'example-demo/cms_datasets/2016/TTToSemiLeptonic/'                   
│                            │     cross_section: 364.31                                                           
│                            │     file_pattern: '*.txt'                                                           
│                            │     tree_name: 'Events'                                                             
│                            │     weight_branch: 'genWeight'                                                      
│                            │     redirector: 'root://xcache/'                                                    
│                            │   }                                                                                 
│                            │   [2]:                                                                              
│                            │ {                                                                                   
│                            │     name: 'ttbar_had'                                                               
│                            │     directory: 'example-demo/cms_datasets/2016/TTToHadronic/'                       
│                            │     cross_section: 380.11                                                           
│                            │     file_pattern: '*.txt'                                                           
│                            │     tree_name: 'Events'                                                             
│                            │     weight_branch: 'genWeight'                                                      
│                            │     redirector: 'root://xcache/'                                                    
│                            │   }                      

In [6]:
# Make a modification
config["datasets"]["max_files"] = 10 # Limit to 1 files per dataset

print("=== SHOW ONLY CHANGES ===")
display_config_table({"datasets": config["datasets"]},
                    expand=True,
                    compare_with={"datasets": original_config["datasets"]},
                    show_only_changes=True)

cli_args = []  # No CLI overrides for this demo
full_config = load_config_with_restricted_cli(config, cli_args)

print("✅ Processed CLI arguments and loaded full configuration")
validated_config = Config(**full_config)  # This is the key validation step!

=== SHOW ONLY CHANGES ===


                                          Configuration: DATASETS (1 changes)                                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━
┃ Key                        ┃ Value                                                               ┃      Status   
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━
│ max_files                  │ 10                                                                  │      CHANGED  
└────────────────────────────┴─────────────────────────────────────────────────────────────────────┴───────────────

✅ Processed CLI arguments and loaded full configuration


# 3. Output Manager & Directory Structure

The framework uses a centralized `OutputDirectoryManager` (`utils/output_manager.py`) to organize all analysis outputs.

In [7]:
from utils.output_manager import OutputDirectoryManager

output_manager = OutputDirectoryManager(
    root_output_dir=validated_config.general.output_dir,
    cache_dir=validated_config.general.cache_dir,
    metadata_dir=validated_config.general.metadata_dir,
    skimmed_dir=validated_config.general.skimmed_dir
)

print("✅ Created OutputDirectoryManager with validated paths")


13:41:43 INFO     Output directory manager initialized with root:                              ]8;id=583280;file:///home/cms-jovyan/integration-challenge/cms/utils/output_manager.py\output_manager.py]8;;\:]8;id=425998;file:///home/cms-jovyan/integration-challenge/cms/utils/output_manager.py#73\73]8;;\
                  /home/cms-jovyan/integration-challenge/cms/example-demo/outputs                                  

✅ Created OutputDirectoryManager with validated paths


## 4. Metadata Extraction

Metadata extraction runs `coffea`'s preprocessor to build analysis workitems, a `coffea`-compatible fileset and a summary of the processed `NanoAOD`s. Its outputs stay in memory, but are also stored in `JSON` files to allow re-runing subsequent steps without pre-processing every time.

In [8]:
from utils.metadata_extractor import NanoAODMetadataGenerator
from utils.datasets import ConfigurableDatasetManager

# Step 1: Create dataset manager instance
dataset_manager = ConfigurableDatasetManager(validated_config.datasets)
# Step 2: Create metadata generator instance
validated_config.general.run_metadata_generation=True

client, cluster = get_client()
metadata_generator = NanoAODMetadataGenerator(
    dataset_manager=dataset_manager,
    output_manager=output_manager,
    dask=(client, cluster),
)
# In the back this uses coffea's preprocess function
metadata = metadata_generator.run()
close_client(client, cluster)

         INFO     Initialized dataset manager with 6 datasets                                        ]8;id=977600;file:///home/cms-jovyan/integration-challenge/cms/utils/datasets.py\datasets.py]8;;\:]8;id=206697;file:///home/cms-jovyan/integration-challenge/cms/utils/datasets.py#38\38]8;;\

(<Client: 'tls://192.168.198.7:8786' processes=0 threads=0, memory=0 B>, None)
Running dask


13:41:44 INFO     Starting metadata generation workflow...                                ]8;id=624178;file:///home/cms-jovyan/integration-challenge/cms/utils/metadata_extractor.py\metadata_extractor.py]8;;\:]8;id=849600;file:///home/cms-jovyan/integration-challenge/cms/utils/metadata_extractor.py#421\421]8;;\

         INFO     Building fileset for process: signal                                    ]8;id=309724;file:///home/cms-jovyan/integration-challenge/cms/utils/metadata_extractor.py\metadata_extractor.py]8;;\:]8;id=838571;file:///home/cms-jovyan/integration-challenge/cms/utils/metadata_extractor.py#168\168]8;;\

         INFO     Building fileset for process: ttbar_semilep                             ]8;id=438759;file:///home/cms-jovyan/integration-challenge/cms/utils/metadata_extractor.py\metadata_extractor.py]8;;\:]8;id=586970;file:///home/cms-jovyan/integration-challenge/cms/utils/metadata_extractor.py#168\168]8;;\

...

13:43:02 INFO     WorkItems metadata saved to                                             ]8;id=599529;file:///home/cms-jovyan/integration-challenge/cms/utils/metadata_extractor.py\metadata_extractor.py]8;;\:]8;id=736959;file:///home/cms-jovyan/integration-challenge/cms/utils/metadata_extractor.py#722\722]8;;\
                  /home/cms-jovyan/integration-challenge/cms/example-demo/outputs/metadat                          
                  a/workitems.json                                                                                 

         INFO     NanoAODs summary generated.                                             ]8;id=324214;file:///home/cms-jovyan/integration-challenge/cms/utils/metadata_extractor.py\metadata_extractor.py]8;;\:]8;id=881214;file:///home/cms-jovyan/integration-challenge/cms/utils/metadata_extractor.py#561\561]8;;\

         INFO     NanoAODs summary written to                                             ]8;id=865599;file:///home/cms-jovyan/integration-challenge/cms/utils/metadata_extractor.py\metadata_extractor.py]8;;\:]8;id=442014;file:///home/cms-jovyan/integration-challenge/cms/utils/metadata_extractor.py#493\493]8;;\
                  /home/cms-jovyan/integration-challenge/cms/example-demo/outputs/metadat                          
                  a/nanoaods.json                                                                                  

         INFO     Metadata generation complete.                                           ]8;id=679283;file:///home/cms-jovyan/integration-challenge/cms/utils/metadata_extractor.py\metadata_extractor.py]8;;\:]8;id=598263;file:///home/cms-jovyan/integration-challenge/cms/utils/metadata_extractor.py#433\433]8;;\

(None, None)

# 5. Skimming & Event Selection

The skimming step processes the workitems from previous step, and applies event and branch selections with `dask` and a `coffea`-like processor (see [Alex's issue](https://github.com/scikit-hep/coffea/issues/1393)). Currently the skimming results are in-memory events, but also stored on disk as `ROOT` files to avoid having to run this multiple times. The events are also cached in `.pkl` files for faster re-reading. Eventually this step needs to be integrated with subsequent steps in a complete `coffea` or `coffea`-like processor.

In [10]:
from utils.skimming import process_workitems_with_skimming

# Extract workitems and fileset from metadata generator
fileset = metadata_generator.fileset
workitems = metadata_generator.workitems

print(f"📊 Processing {len(workitems)} workitems across {len(fileset)} datasets")

# Disable caching for demonstration
validated_config.general.read_from_cache = False
validated_config.general.run_skimming = True

client, cluster = get_client()
# Skim data with dask according to the workitems
processed_datasets = process_workitems_with_skimming(
    workitems,
    validated_config,
    output_manager,
    fileset,
    metadata_generator.nanoaods_summary
)
close_client(client, cluster)

# Display the structure of processed datasets
print(f"\n✅ Skimming complete! Processed datasets structure x:")
for dataset_name, events_list in processed_datasets.items():
    print(f"  📁 {dataset_name} processed")
    for i, (events, metadata) in enumerate(events_list):
        print(f"    └── File {i+1}: {len(events)} events, {len(events.fields)} branches")

print(f"\n💾 Skimmed data saved to: {output_manager.get_skimmed_dir()}")

📊 Processing 241 workitems across 4 datasets


13:45:01 INFO     Starting workitem preprocessing with 241 workitems                                ]8;id=425024;file:///home/cms-jovyan/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=938042;file:///home/cms-jovyan/integration-challenge/cms/utils/skimming.py#705\705]8;;\

         INFO     Initialized workitem-based skimming manager                                       ]8;id=588940;file:///home/cms-jovyan/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=970699;file:///home/cms-jovyan/integration-challenge/cms/utils/skimming.py#372\372]8;;\

         INFO     Running skimming                                                                  ]8;id=403351;file:///home/cms-jovyan/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=551386;file:///home/cms-jovyan/integration-challenge/cms/utils/skimming.py#720\720]8;;\

         INFO     Processing 241 workitems                                                          ]8;id=998403;file:///home/cms-jovyan/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=567008;file:///home/cms-jovyan/integration-challenge/cms/utils/skimming.py#403\403]8;;\

...

         INFO     Cached events for ttbar_semilep__nominal                                          ]8;id=609821;file:///home/cms-jovyan/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=232002;file:///home/cms-jovyan/integration-challenge/cms/utils/skimming.py#831\831]8;;\

         INFO     Found existing skimmed files for signal__nominal: 1                               ]8;id=779420;file:///home/cms-jovyan/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=710925;file:///home/cms-jovyan/integration-challenge/cms/utils/skimming.py#524\524]8;;\

         INFO     Merged 1 files → 19326 events for signal__nominal                                 ]8;id=641143;file:///home/cms-jovyan/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=899593;file:///home/cms-jovyan/integration-challenge/cms/utils/skimming.py#822\822]8;;\

         INFO     Cached events for signal__nominal                                                 ]8;id=572959;file:///home/cms-jovyan/integration-challenge/cms/utils/skimming.py\skimming.py]8;;\:]8;id=165284;file:///home/cms-jovyan/integration-challenge/cms/utils/skimming.py#831\831]8;;\


✅ Skimming complete! Processed datasets structure x:
  📁 ttbar_lep__nominal processed
    └── File 1: 394079 events, 10 branches
  📁 ttbar_had__nominal processed
    └── File 1: 8495 events, 10 branches
  📁 ttbar_semilep__nominal processed
    └── File 1: 181492 events, 10 branches
  📁 signal__nominal processed
    └── File 1: 19326 events, 10 branches

💾 Skimmed data saved to: /home/cms-jovyan/integration-challenge/cms/example-demo/outputs/skimmed


In [ ]:
# Enable caching to show how it speeds up subsequent runs
validated_config.general.read_from_cache = True
validated_config.general.run_skimming = False
# Run the skimming again - it will look for cached files
# if they don't exist, it will fallback to skim regularly
cached_datasets = process_workitems_with_skimming(
    workitems,
    validated_config,
    output_manager,
    fileset,
    metadata_generator.nanoaods_summary
)

# Show cache directory contents
import os
cache_files = os.listdir(output_manager.get_skimmed_dir())
print(f"💾 Cached files in skimmed directory: {len(cache_files)} files")
print(f"📁 Cache location: {output_manager.get_cache_dir()}")

# 6. Analysis & Histogramming

This step encapsulates a few underlying function calls:
1. Global event selection is applied
2. If MVA training is configured and enabled, an MVA training will be triggered
3. Apply corrections from `correctionlib`
4. Ghost observables will be computed and added to event record
6. Channels will be built with their corresponding selections
7. Compute all observables (once nomina, once per systematic variation)
8. Create histograms

In [11]:
# Import the analysis class for the analysis
from analysis.nondiff import NonDiffAnalysis
from utils.output_files import save_histograms_to_root

validated_config.general.run_histogramming=True

# Initialize the analysis object
# This analysis object will handle histogram creation, observable computation, and systematic variations
nondiff_analysis = NonDiffAnalysis(validated_config, processed_datasets, output_manager)

print(f"🔬 Analysis initialized for {len(nondiff_analysis.processed_datasets)} datasets")

# Loop over each dataset and its associated event data
for dataset_name, events_list in nondiff_analysis.processed_datasets.items():
    print(f"📊 Processing dataset: {dataset_name}")
    # Each dataset contains a list of (events, metadata) tuples (in case of chunking)
    # Currently merging of chunks is default --> single tuple per dataset
    for events, metadata in events_list:
        print(f"   • Processing {len(events)} events with metadata: {metadata['process']}")
        # Process the events through the analysis pipeline:
        nondiff_analysis.process(events, metadata)
        
save_histograms_to_root(
    nondiff_analysis.nD_hists_per_region,
    output_file=nondiff_analysis.output_manager.get_histograms_dir() / "histograms.root",
)

print(f"📈 Generated histograms for channels: {[ch.name for ch in validated_config.channels]}")

🔬 Analysis initialized for 4 datasets
📊 Processing dataset: ttbar_lep__nominal
   • Processing 394079 events with metadata: ttbar_lep


13:47:48 INFO     Applying selection for CMS_WORKSHOP in ttbar_lep with variation nominal            ]8;id=633874;file:///home/cms-jovyan/integration-challenge/cms/analysis/nondiff.py\nondiff.py]8;;\:]8;id=769587;file:///home/cms-jovyan/integration-challenge/cms/analysis/nondiff.py#154\154]8;;\

         INFO     Number of weighted events in CMS_WORKSHOP: 71.21                                   ]8;id=772215;file:///home/cms-jovyan/integration-challenge/cms/analysis/nondiff.py\nondiff.py]8;;\:]8;id=365759;file:///home/cms-jovyan/integration-challenge/cms/analysis/nondiff.py#205\205]8;;\

         INFO     Number of raw events in CMS_WORKSHOP: 327                                          ]8;id=89713;file:///home/cms-jovyan/integration-challenge/cms/analysis/nondiff.py\nondiff.py]8;;\:]8;id=341038;file:///home/cms-jovyan/integration-challenge/cms/analysis/nondiff.py#208\208]8;;\

         INFO     Computing observable workshop_mtt                                                  ]8;id=700472;file:///home/cms-jovyan/integration-challenge/cms/analysis/nondiff.py\nondiff.py]8;;\:]8;id=147186;file:///home/cms-jovyan/integration-challenge/cms/analysis/nondiff.py#213\213]8;;\

...

         INFO     Number of weighted events in CMS_WORKSHOP: 3156.00                                 ]8;id=728544;file:///home/cms-jovyan/integration-challenge/cms/analysis/nondiff.py\nondiff.py]8;;\:]8;id=935890;file:///home/cms-jovyan/integration-challenge/cms/analysis/nondiff.py#205\205]8;;\

         INFO     Number of raw events in CMS_WORKSHOP: 3214                                         ]8;id=317126;file:///home/cms-jovyan/integration-challenge/cms/analysis/nondiff.py\nondiff.py]8;;\:]8;id=285235;file:///home/cms-jovyan/integration-challenge/cms/analysis/nondiff.py#208\208]8;;\

         INFO     Computing observable workshop_mtt                                                  ]8;id=677834;file:///home/cms-jovyan/integration-challenge/cms/analysis/nondiff.py\nondiff.py]8;;\:]8;id=321898;file:///home/cms-jovyan/integration-challenge/cms/analysis/nondiff.py#213\213]8;;\

         INFO     Histograms successfully written to ROOT file                                  ]8;id=417275;file:///home/cms-jovyan/integration-challenge/cms/utils/output_files.py\output_files.py]8;;\:]8;id=281511;file:///home/cms-jovyan/integration-challenge/cms/utils/output_files.py#147\147]8;;\
                  /home/cms-jovyan/integration-challenge/cms/example-demo/outputs/histograms/hi                    
                  stograms.root                                                                                    

📈 Generated histograms for channels: ['CMS_WORKSHOP']


# 7. Statistical Analysis

The statistical analysis step performs a fit from the histograms and visualize results. This uses `cabinetry` with a manually created config. It also creates and stores the corresponding `pyhf` workspace.

**Note: the following fit is performed with the full CMS open-data dataset, not the small example here** 

In [13]:
import cabinetry
validated_config.statistics.cabinetry_config = "./example-demo/outputs/cabinetry/cabinetry_config.yaml"
cabinetry_config = cabinetry.configuration.load(
    validated_config.statistics.cabinetry_config
)
data, fit_results, pre_fit_predictions, postfit_predictions = (
    nondiff_analysis.run_fit(cabinetry_config=cabinetry_config)
)
cabinetry.visualize.data_mc(
    pre_fit_predictions,
    data,
    close_figure=False,
    config=cabinetry_config,
    figure_folder=nondiff_analysis.output_manager.get_statistics_dir(),

)
cabinetry.visualize.data_mc(
    postfit_predictions,
    data,
    close_figure=False,
    config=cabinetry_config,
    figure_folder=nondiff_analysis.output_manager.get_statistics_dir(),
)
cabinetry.visualize.pulls(fit_results, close_figure=False, figure_folder=nondiff_analysis.output_manager.get_statistics_dir(),)

13:51:35 INFO     opening config file example-demo/outputs/cabinetry/cabinetry_config.yaml      ]8;id=712403;file:///usr/local/lib/python3.12/site-packages/cabinetry/configuration.py\configuration.py]8;;\:]8;id=606522;file:///usr/local/lib/python3.12/site-packages/cabinetry/configuration.py#26\26]8;;\

KeyInFileError: not found: 'CMS_WORKSHOP__workshop_mtt__data' (with any cycle number)

    Available keys: 'CMS_WORKSHOP__workshop_mtt__signal;1', 'CMS_WORKSHOP__workshop_mtt__ttbar_lep;1', 'CMS_WORKSHOP__workshop_mtt__ttbar_had;1', 'CMS_WORKSHOP__workshop_mtt__ttbar_semilep;1', 'CMS_WORKSHOP__workshop_mtt__signal__pu_weight_up;1'...

in file example-demo/outputs/histograms/histograms.root

## 8. Full Workflow

The above workflow is broken down for demonstration, but the simplest way to use it is by putting the main pieces together in one simple steering script and running it from the command-line with CLI overrides of config options. The current implementation is steered by `analysis.py` and can be simply executed with:

```sh
python analysis.py
```

---